In [6]:
#!/usr/bin/env python3
"""
Arithmetic-estimate validation with user-set plateau rule
────────────────────────────────────────────────────────
Edit WINDOW_SEC and TOL_PLATEAU below, then run.

Outputs:
  • Excel file time_to_steady_arithmetic.xlsx
  • Console statistics comparing T_num vs T_est_ari
"""

import itertools, numpy as np, pandas as pd, warnings
from math import sin, cos, pi
from scipy.integrate import solve_ivp, quad, IntegrationWarning
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# ───────────────────────── Plateau rule (EDIT HERE) ─────────────────────────
WINDOW_SEC   = 5          # seconds that δ(t) must stay flat
TOL_PLATEAU  = 1e-2        # ± band (rad) defining “flat”
# ────────────────────────────────────────────────────────────────────────────

# Fixed numerical parameters
T_MAX, DT = 100.0, 0.001
TIME_EVAL = np.arange(0, T_MAX + DT, DT)
WIN = int(WINDOW_SEC / DT)
MIN_INTERVAL = 1e-3

# Parameter grid (same 400 comb. as before)
p1_vals = np.linspace(0, 2, 10)
p2_vals = np.linspace(0, 2, 10)
a_vals  = [1.0, 3.0]
b_vals  = [1.0, 2.0]
PARAM_GRID = list(itertools.product(p1_vals, p2_vals, a_vals, b_vals))

# ───────────── ODE & f(δ) ─────────────
def single_cell_ode(t, y, p1, p2, a, b):
    θ, ψ = y
    sin2, sin_, cos_ = sin(2*(ψ-θ)), sin(ψ-θ), cos(ψ-θ)
    denom = 2*((a*sin_)**2 + (b*cos_)**2)**1.5
    num   = (a*b)*(a**2 - b**2)*sin2
    return [-1.0 - p1*num/denom,  -p2*sin2]

def f_delta(δ, p1, p2, a, b):
    sin2, sin_, cos_ = np.sin(2*δ), np.sin(δ), np.cos(δ)
    denom = 2*((a*sin_)**2 + (b*cos_)**2)**1.5
    num   = (a*b)*(a**2 - b**2)*sin2
    return -p2*sin2 - (-1.0 - p1*num/denom)

# ───────────── helper: zeros in one π-period ─────────────
def one_period_zeros(p1,p2,a,b,n_scan=20000):
    x=np.linspace(-pi,pi,n_scan); y=f_delta(x,p1,p2,a,b)
    roots=[]
    for i in range(n_scan-1):
        if y[i]*y[i+1]<0:
            lo,hi=x[i],x[i+1]
            for _ in range(30):
                mid=.5*(lo+hi)
                (hi,lo)=(mid,lo) if f_delta(lo,p1,p2,a,b)*f_delta(mid,p1,p2,a,b)<0 else (hi,mid)
            roots.append(.5*(lo+hi))
    roots.sort()
    if not roots: return []
    start=roots[0]; per=[z for z in roots if start<z<start+pi]; end=start+pi
    if abs(end-per[-1])>1e-8: per.append(end)
    per.insert(0,start); return per

# ───────────── integrate until plateau ─────────────
def integrate_to_steady(δ0,p1,p2,a,b):
    θ0,ψ0 = pi/3, pi/3 + δ0
    sol=solve_ivp(lambda t,y: single_cell_ode(t,y,p1,p2,a,b),
                  (0,T_MAX),[θ0,ψ0],t_eval=TIME_EVAL,
                  rtol=1e-6,atol=1e-9)
    if not sol.success: return None,None
    δ=sol.y[1]-sol.y[0]
    for i in range(len(δ)-WIN):
        win=δ[i:i+WIN]
        if np.max(np.abs(win-win[0])) < TOL_PLATEAU:
            return win[0], sol.t[i]
    return None,None

# ───────────── worker ─────────────
def run_param(params):
    p1,p2,a,b=params
    zeros=one_period_zeros(p1,p2,a,b)
    if len(zeros)<2: return []
    out=[]
    for zL,zR in zip(zeros[:-1],zeros[1:]):
        Δ_int=zR-zL
        if Δ_int<MIN_INTERVAL: continue
        for q in (0.25,0.5,0.75):
            δ0=zL+q*Δ_int
            sgn=np.sign(f_delta(δ0,p1,p2,a,b))
            if sgn==0: continue
            δ_star = zR if sgn>0 else zL
            Δ=abs(δ_star-δ0)
            I,_ = quad(lambda x: abs(f_delta(x,p1,p2,a,b)),
                       min(δ0,δ_star), max(δ0,δ_star), limit=200)
            if I<1e-8: continue
            T_est = Δ**2 / I
            steady_val,T_num = integrate_to_steady(δ0,p1,p2,a,b)
            if steady_val is None: continue
            out.append(dict(T_num=T_num, T_est=T_est))
    return out

# ───────────── main ─────────────
def main():
    with Pool(cpu_count()) as pool:
        rows=[]
        for res in tqdm(pool.imap_unordered(run_param, PARAM_GRID),
                        total=len(PARAM_GRID), desc="Running"):
            rows.extend(res)
    df=pd.DataFrame(rows)
    df.to_excel("time_to_steady_arithmetic.xlsx", index=False)

    diff = df["T_num"] - df["T_est"]
    med_abs = diff.abs().median(); q1,q3 = diff.abs().quantile([0.25,0.75])
    med_rel = (diff.abs()/df["T_num"]).median()
    within2 = ((df["T_est"]/df["T_num"]).between(0.5,2)).mean()
    n_gt = (diff > 1e-6).sum(); n_lt=(diff<-1e-6).sum(); N=len(df)

    print(f"\nSaved → time_to_steady_arithmetic.xlsx  ({N} trajectories)")
    print(f"Plateau rule: window {WINDOW_SEC} s, tolerance ±{TOL_PLATEAU}")
    print("\nResults for arithmetic estimator")
    print(f"T_num > T_est : {n_gt}/{N}  ({100*n_gt/N:.2f} %)")
    print(f"T_num < T_est : {n_lt}/{N}  ({100*n_lt/N:.2f} %)")
    print(f"Median |Δt|    = {med_abs:.4f} s  (IQR {q1:.4f}–{q3:.4f})")
    print(f"Median rel err = {100*med_rel:.2f} %")
    print(f"Within factor-2= {100*within2:.2f} %")

if __name__ == "__main__":
    # silence quad warnings (optional)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=IntegrationWarning)
        main()


Running: 100%|██████████| 400/400 [00:04<00:00, 84.52it/s] 



Saved → time_to_steady_arithmetic.xlsx  (1602 trajectories)
Plateau rule: window 5 s, tolerance ±0.01

Results for arithmetic estimator
T_num > T_est : 1575/1602  (98.31 %)
T_num < T_est : 27/1602  (1.69 %)
Median |Δt|    = 0.6477 s  (IQR 0.4119–0.9602)
Median rel err = 43.50 %
Within factor-2= 63.42 %
